In [ ]:
# drop with too many NA's
# Need to work out which variable are correlated to drop
# Need to see if any variables are duplicated
# Need to see if any variables are constant for both above and under 50K

In [1]:
import pandas as pd
import numpy as np
from config import config
import matplotlib.pyplot as plt
import preprocessors
from helpers import get_meta_columns
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
# Reading in the data
data = pd.read_csv(config.TRAIN_DIR, names=get_meta_columns(), na_values=' ?')

# encodes the target variable as binary
data[config.TARGET] = data[config.TARGET].map(config.TARGET_ENCODING)

X_train, X_test, y_train, y_test = train_test_split(
        data[config.FEATURES], data[config.TARGET], test_size=0.2, random_state=0
        )  

income_pipeline = Pipeline(
    [
        (
            "EncodeNotInUniverse",
            preprocessors.EncodeNotInUniverse(variables=config.DROP_VALS),
        ),    
        (
            "DropNaFeatures",
            preprocessors.DropNaFeatures(variables=config.DROP_VALS),
        ),
        (
            "Fill_NA_encoder",
            preprocessors.FillNAEncoder(variables=config.CONTAINS_NAN_VALS),
        ),
        (
            "categorical_encoder",
            preprocessors.CategoricalEncoder(variables=config.CATEGORICAL_VALS[1:]),
        ),
         (
            "Ordinal Encoder",
            preprocessors.OrdinalEncoder(variables=config.CATEGORICAL_VALS[0]),
        ),       
        (
            "Numeric_variable_scalar", 
            preprocessors.ScaleNumeric(variables=config.NUMERIC_VALS),
        ),
    ]
        )  

X_train_processed =income_pipeline.fit_transform(
                                                    X_train[config.FEATURES], y_train)




In [7]:
# ordered by level of education increasing
data['education'] = data['education'].map(education_dict)


# Numeric values
# scales numeric values
# no na values in numeric
mm_scalar = MinMaxScaler()
data[config.NUMERIC_VALS] = mm_scalar.fit_transform(data[config.NUMERIC_VALS])

education                                     0
marital stat                                  0
major industry code                           0
race                                          0
hispanic origin                               0
sex                                           0
full or part time employment stat             0
tax filer stat                                0
detailed household and family stat            0
detailed household summary in household       0
live in this house 1 year ago                 0
country of birth father                    6713
country of birth mother                    6119
country of birth self                      3393
citizenship                                   0
dtype: int64

In [3]:
def analysis_categoricial(df, var):
    ploting_df = pd.DataFrame(data=df[['Income',var]].value_counts())
    ploting_df = ploting_df.reset_index()
    sns.catplot(x=var,y=0,hue='Income', data = ploting_df, kind='bar')
    plt.show()

for var in config.CATEGORICAL_VALS:
    analysis_categoricial(data, var)

# Cardinality check
for var in config.CATEGORICAL_VALS:
    print(var, len(data[var].unique()))

education 17
marital stat 7
major industry code 24
race 5
hispanic origin 10
sex 2
full or part time employment stat 8
tax filer stat 6
detailed household and family stat 38
detailed household summary in household 8
live in this house 1 year ago 3
country of birth father 43
country of birth mother 43
country of birth self 43
citizenship 5
